# Parameterized Quantum Circuits for Antisymmetric Matrices

## Install core librariries

In [ ]:
!pip install qiskit
!pip install qiskit[visualization]
!pip install qiskit-aer

## Core functions

In [ ]:
# -*- coding: utf-8 -*-
"""parameterized circuit.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1cnxf1ON6PCv7UIBCAa58fTDakN6mgAzv
"""

#!pip install qiskit
#!pip install qiskit[visualization]
#!pip install qiskit-aer

import numpy as np
from qiskit_aer import Aer
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import ParameterVector
from scipy.optimize import minimize
from scipy.stats import unitary_group
from scipy.linalg import expm, logm, norm
import matplotlib.pyplot as plt
from scipy.linalg import expm as scipyexpm
from scipy.linalg import logm as scipylogm
from scipy.linalg import norm as scipynorm


def generate_random_antisymmetric_matrix(n, rng=None):

    if rng == None:
        rng = np.random.default_rng()
    # Initialize a matrix with zeros
    A = np.zeros((n, n))
    # Fill the upper triangle with random + or - reals
    for i in range(n):
        for j in range(i + 1, n):
            A[i, j] = rng.uniform(-1, 1)
    # Fill the lower triangle to make it antisymmetric
    for i in range(n):
        for j in range(i):
            A[i, j] = -A[j, i]
    return A


def generate_random_antisymmetric_ones(n, rng=None):
    if rng == None:
        rng = np.random.default_rng()
    # Initialize a matrix with zeros
    A = np.zeros((n, n))
    # Fill the upper triangle with random +1 or -1
    for i in range(n):
        for j in range(i + 1, n):
            A[i, j] = rng.choice([-1, 1])
    # Fill the lower triangle to make it antisymmetric
    for i in range(n):
        for j in range(i):
            A[i, j] = -A[j, i]
    return A

def generate_random_antisymmetric_ones_with_zeros(n, rng=None):
    if rng == None:
        rng = np.random.default_rng()
    # Initialize a matrix with zeros
    A = np.zeros((n, n))
    # Fill the upper triangle with random +1 or -1
    for i in range(n):
        for j in range(i + 1, n):
            A[i, j] = rng.choice([-1, 1,0])
    # Fill the lower triangle to make it antisymmetric
    for i in range(n):
        for j in range(i):
            A[i, j] = -A[j, i]
    return A

def generate_signed_permutation_matrix(n):
    """Generates a random signed permutation matrix of size n x n."""

    # Generate a random permutation of the rows
    permutation = np.random.permutation(n)
    # Randomly assign signs to each element of the diagonal
    diagonal_matrix = np.diag(np.random.choice([-1, 1], size=n))
    P = diagonal_matrix[permutation, :]

    return P


def create_parameterized_P(n_qubits):
    """Create a parameterized quantum circuit with 4 alternating layers"""
    qc = QuantumCircuit(n_qubits)

    # Calculate total number of parameters
    layer2_gates = n_qubits - 1  # CRy in first ladder
    layer4_gates = max(n_qubits - 2, 0)  # CRy in second ladder
    total_params = 2 * n_qubits + layer2_gates + layer4_gates

    params = ParameterVector("θ_P", total_params)
    param_idx = 0

    # Layer 1: Single Ry gates
    for q in range(n_qubits):
        qc.ry(params[param_idx], q)
        param_idx += 1

    # Layer 2: CRy ladder (0→1, 2→3, ...)
    for control in range(0, n_qubits - 1, 2):
        qc.cry(params[param_idx], control, control + 1)
        param_idx += 1

    # Layer 3: Single Ry gates
    for q in range(n_qubits):
        qc.ry(params[param_idx], q)
        param_idx += 1

    # Layer 4: CRy ladder starting from qubit 1
    for control in range(1, n_qubits - 1, 2):
        qc.cry(params[param_idx], control, control + 1)
        param_idx += 1

    return qc, params


def create_parameterized_qft(n_qubits):
    """Creates a QFT circuit with parameterized rotation gates."""
    qc = QuantumCircuit(n_qubits)
    params = ParameterVector(
        "θ_F", n_qubits * (n_qubits - 1) // 2
    )  # Total params for controlled rotations

    param_idx = 0

    for i in range(n_qubits):
        qc.h(i)  # Apply Hadamard (fixed)

        # Apply controlled rotations with trainable parameters
        for j in range(i + 1, n_qubits):
            # Parameterized CRZ (could also use CU1 or other phase gates)
            angle = params[param_idx]
            qc.cp(angle, j, i)  # Controlled-phase (replaces fixed QFT angles)
            param_idx += 1

    return qc, params


def create_parameterized_kronecker_rz_D(n_qubits):
    """
    Creates a circuit representing the Kronecker product of Rz gates.
    Each qubit gets an independent Rz rotation with a trainable parameter.

    Args:
        n_qubits (int): Number of qubits.

    Returns:
        QuantumCircuit: Parameterized circuit.
        ParameterVector: List of parameters (θ_0, θ_1, ..., θ_{n-1}).
    """
    qc = QuantumCircuit(n_qubits)
    params = ParameterVector("θ_D", n_qubits)  # One parameter per qubit

    for qubit in range(n_qubits):
        qc.rz(params[qubit], qubit)  # Apply Rz(θ_i) to each qubit

    return qc, params


def create_parameterized_kronecker_rz_cot_Lambda(n_qubits, angle_scale=1):
    """
    Creates a circuit with Rz gates whose angles are cotangent-parameterized.

    Args:
        n_qubits (int): Number of qubits.
        angle_scale (float): Scaling factor for the angle denominator (default: 1).

    Returns:
        QuantumCircuit: Parameterized circuit.
        ParameterVector: List of underlying parameters (α_0, α_1, ..., α_{n-1}).
    """
    qc = QuantumCircuit(n_qubits)
    alpha_params = ParameterVector("α_Λ", n_qubits)  # Raw parameters before cotangent

    for qubit in range(n_qubits):
        # Compute θ_i = cot(α_i * π / (2^angle_scale))
        theta = np.pi / (2**angle_scale) * alpha_params[qubit]
        qc.rz(1 / np.tan(theta), qubit)

    return qc, alpha_params


def create_combine_circuit(n_qubits=3):
    """Assemble the full circuit from its components."""
    # Create first circuit (Ry + CRy ladder structure)
    qcP, paramsP = create_parameterized_P(n_qubits)

    # Create circuit D = Rz⊗ Rz ⊗...
    qcD, paramsD = create_parameterized_kronecker_rz_D(n_qubits)

    # Create parameterized qft
    qcF, paramsF = create_parameterized_qft(n_qubits)

    # Create second circuit (Rx, Ry, Rz structure)
    qcLambda, paramsLambda = create_parameterized_kronecker_rz_cot_Lambda(
        n_qubits, angle_scale=1
    )

    # the combined circuit is P D F Lambda F^{-1} D^{-1} P^{-1}.

    # Create inverse circuits (adjoints)
    qcP_inv = qcP.inverse()
    qcD_inv = qcD.inverse()
    qcF_inv = qcF.inverse()

    # Combine circuits: P → D → F → Λ → F† → D† → P†
    qc = qcP.compose(qcD)
    qc = qc.compose(qcF)
    qc = qc.compose(qcLambda)
    qc = qc.compose(qcF_inv)
    qc = qc.compose(qcD_inv)
    qc = qc.compose(qcP_inv)
    # Collect all parameters from the combined circuit
    params = list(qc.parameters)

    return qc, params


def loss_commutator(Uestimate, Atarget):
    # Not working well
    """Loss = ||A Uestimate - Uestimate A||_F"""
    return np.linalg.norm(
        Atarget @ np.asarray(Uestimate) - np.asarray(Uestimate) @ Atarget, "fro"
    )


def loss_log_frobenius(Uestimate, Atarget):
    """Loss = ||log(Uestimate) - A||_F"""
    # todo:We can find this from the circuit by using matrix multiplications
    Aestimate = scipylogm(Uestimate)
    return np.linalg.norm(Aestimate - Atarget, "fro")


def calculate_loss(
    simulator,
    compiled_qc,
    params,
    target_unitary,
    param_values,
    loss_function=loss_log_frobenius,
):
    """
    Given the pre-transpiled circuit, bind new parameter values,
    simulate the resulting circuit unitary, and compute the fidelity loss.
    """
    # Bind parameters to the pre-transpiled circuit
    bound_qc = compiled_qc.assign_parameters(dict(zip(params, param_values)))
    # Run the simulation; get_unitary() typically returns a NumPy array directly.
    unitary = simulator.run(bound_qc).result().get_unitary()

    return loss_function(unitary, target_unitary)



## Run optimization

In [ ]:

n_qubits = 2
max_iter = 200

qc, params = create_combine_circuit(n_qubits)

# TODO: Initialize random parameters
init_params = np.random.uniform(0, 2 * np.pi, len(params))


# --- Preprocessing for efficiency ---
# 1. Create and cache the simulator backend.
simulator = Aer.get_backend("unitary_simulator")

# 2. Pre-transpile the circuit once so you don't have to redo it on every iteration.
compiled_qc = transpile(qc, simulator)

"""Optimization routine to approximate random unitary"""
# Generate target unitary
rng = np.random.default_rng()
Atarget = generate_random_antisymmetric_matrix(2**n_qubits, rng)
target = Atarget

# Optimization with plotting
loss_function = loss_log_frobenius

loss_history = []


def callback(xk):
    loss = calculate_loss(
        simulator, compiled_qc, params, target, xk, loss_function=loss_function
    )
    loss_history.append(loss)
    # print(f"Iteration: Current loss = {loss}")
    return False


result = minimize(
    lambda x: calculate_loss(
        simulator, compiled_qc, params, target, x, loss_function=loss_function
    ),
    init_params,
    method="L-BFGS-B",
    callback=callback,  # for history
    options={"maxiter": max_iter},
)

# Plot results
plt.plot(loss_history)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()


print(f"Optimization {'succeeded' if result.success else 'failed'}")
print(f"Final loss: {result.fun:.4f}")
print(f"Optimized parameters: {result.x}")

# --- Verify the final unitary from the optimized circuit ---
bound_qc_final = compiled_qc.assign_parameters(result.x)
Uestimate = simulator.run(bound_qc_final).result().get_unitary()

Utarget = scipyexpm(Atarget)
Aestimate = scipylogm(Uestimate)

evalAestimate, evecAestimate = np.linalg.eig(Aestimate)
evalAtarget, evecAtarget = np.linalg.eig(Atarget)

print("Eigenvalues of Aestimate:\n", np.round(np.sort(np.abs(evalAestimate)), 2))
print("Eigenvalues of Atarget:\n", np.round(np.sort(np.abs(evalAtarget)), 2))
print("estimated unitary:\n", np.round(Uestimate, 2))
print("Target unitary:\n", np.round(Utarget, 2))
print("Fidelity of unitaries:\n", np.round(np.asarray(Uestimate) @ Utarget, 2))

Aestimate = scipylogm(Uestimate)
print("Final A:\n", np.round(Aestimate, 2))
print("Target A:\n", np.round(Atarget, 2))
print("Difference of A-Aestimate:\n", np.round(Aestimate - Atarget, 2))


### Output Circuit

In [ ]:
bound_qc_final.draw("mpl")

## Multiple running optimization and drawing figures

In [ ]:

def run_optimization(n_qubits, max_iter, seed=None, label=None):
    """Run a single optimization and return the loss history"""
    qc, params = create_combine_circuit(n_qubits)

    # Initialize random parameters
    if seed is not None:
        np.random.seed(seed)
    init_params = np.random.uniform(0, 2 * np.pi, len(params))

    # Preprocessing for efficiency
    simulator = Aer.get_backend("unitary_simulator")
    compiled_qc = transpile(qc, simulator)

    # Generate target unitary
    rng = np.random.default_rng()  # Fix seed for consistent target
    Atarget = generate_random_antisymmetric_matrix(2**n_qubits, rng)
    target = Atarget

    # Optimization with plotting
    loss_function = loss_log_frobenius
    loss_history = []

    def callback(xk):
        loss = calculate_loss(
            simulator, compiled_qc, params, target, xk, loss_function=loss_function
        )
        loss_history.append(loss)
        return False

    result = minimize(
        lambda x: calculate_loss(
            simulator, compiled_qc, params, target, x, loss_function=loss_function
        ),
        init_params,
        method="L-BFGS-B",
        callback=callback,
        options={"maxiter": max_iter},
    )

    return loss_history, result

def plot_multiple_runs_old(n_qubits=3, max_iter=500, num_runs=5):
    """Run multiple optimizations and plot their loss histories together"""
    plt.figure(figsize=(10, 6))

    for i in range(num_runs):
        loss_history, result = run_optimization(n_qubits, max_iter, seed=i)
        plt.plot(loss_history, label=f'Run {i+1} (Final: {result.fun:.4f})')

    plt.xlabel("Iteration")
    plt.ylabel("Loss")
    plt.title(f"Loss History for {num_runs} Runs (number of qubits = {n_qubits})")
    # plt.legend()
    plt.grid(True)
    plt.show()

def plot_multiple_runs(n_qubits=3, max_iter=500, num_runs=5):
    """Run multiple optimizations and plot their loss histories together"""
    # Set LaTeX-compatible font settings
    plt.rcParams.update({
        "text.usetex": False,
        "font.size": 12,
        "axes.titlesize": 14,
        "axes.labelsize": 12,
        "legend.fontsize": 10,
        "xtick.labelsize": 10,
        "ytick.labelsize": 10,
        "figure.dpi": 300
    })

    fig, ax = plt.subplots(figsize=(6, 4))  # Better aspect ratio for papers

    # Create a color palette with better distinction
    colors = plt.cm.viridis(np.linspace(0, 1, num_runs))

    for i in range(num_runs):
        loss_history, result = run_optimization(n_qubits, max_iter, seed=i)
        ax.plot(loss_history,
                #color=colors[i],
                linewidth=1.5,
                linestyle='-',
                #marker='' if len(loss_history) > 100 else '.',
                #markersize=4,
                label=fr'Run {i+1}: L={result.fun:.2f}')

    # Axis labels with proper LaTeX formatting
    ax.set_xlabel(r"Iteration Number ($k$)", labelpad=10)
    ax.set_ylabel(r"Loss $L(\theta_k)$", labelpad=10)

    # Title with anti-symmetry mention (modify as needed)
    ax.set_title(rf"Optimization Trajectories ($n_q={n_qubits}$)", pad=15)

    # Configure grid and spines
    ax.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # # Legend formatting
    # legend = ax.legend(loc='upper right',
    #                   frameon=True,
    #                   fancybox=True,
    #                   framealpha=0.9,
    #                   edgecolor='black')
    # legend.get_frame().set_linewidth(0.5)

    # Axis limits and tick formatting
    ax.set_xlim(left=0)
    ax.set_yscale('log') if max(loss_history) > 10 else ax.set_yscale('linear')

    # Adjust layout and save
    plt.tight_layout(pad=2.0)
    plt.savefig("n5real.png", bbox_inches='tight')
    plt.show()
# Example usage:
plot_multiple_runs(n_qubits=2, max_iter=200, num_runs=30)